In [ ]:
from pathlib import Path
import pandas as pd

def sniff_and_read(path, nbytes=4000):
    p = Path(path)
    raw = p.read_bytes()[:nbytes]

    bom = {
        b'\xff\xfe': 'utf-16le',
        b'\xfe\xff': 'utf-16be',
        b'\xef\xbb\xbf': 'utf-8-sig'
    }
    for sig, enc in bom.items():
        if raw.startswith(sig):
            print(f"[BOM] {p.name} -> {enc}")
            return pd.read_csv(p, encoding=enc, sep=None, engine="python")

    candidates = ["utf-8", "cp949", "euc-kr", "utf-16", "utf-16le", "utf-16be", "latin1"]
    last_err = None
    for enc in candidates:
        try:
            df = pd.read_csv(p, encoding=enc, sep=None, engine="python")
            print(f"[OK] {p.name} -> encoding={enc} (rows={len(df)}, cols={df.shape[1]})")
            return df
        except Exception as e:
            last_err = e

    raise last_err

library_info_path = "../data/raw/전국공공도서관정보.csv"

df_info = sniff_and_read(library_info_path)
df_std  = sniff_and_read(library_standard_path)

print("\n=== 전국공공도서관정보 ===")
df_info.info()
print(df_info.columns.tolist())

[OK] 전국공공도서관정보.csv -> encoding=cp949 (rows=1296, cols=11)
[OK] 전국도서관표준데이터.csv -> encoding=utf-8 (rows=3546, cols=28)

=== 전국공공도서관정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   평가년도       1296 non-null   int64  
 1   도서관구분      1296 non-null   object 
 2   도서관코드      1296 non-null   int64  
 3   도서관명       1296 non-null   object 
 4   행정구역       1296 non-null   object 
 5   시군구        1296 non-null   object 
 6   장서수(인쇄)    1296 non-null   int64  
 7   사서수        1296 non-null   float64
 8   대출자수       1296 non-null   int64  
 9   대출권수       1296 non-null   int64  
 10  자료구입비(천원)  1296 non-null   int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 111.5+ KB
['평가년도', '도서관구분', '도서관코드', '도서관명', '행정구역', '시군구', '장서수(인쇄)', '사서수', '대출자수', '대출권수', '자료구입비(천원)']

=== 전국도서관표준데이터 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3546 e

In [9]:
import pandas as pd

df_std = pd.read_csv(
    "../data/raw/전국도서관표준데이터.csv",
    encoding="cp949"
)

df_std.info()
df_std.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3546 entries, 0 to 3545
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   도서관명        3546 non-null   object 
 1   시도명         3546 non-null   object 
 2   시군구명        3546 non-null   object 
 3   도서관유형       3546 non-null   object 
 4   휴관일         3546 non-null   object 
 5   평일운영시작시각    3546 non-null   object 
 6   평일운영종료시각    3546 non-null   object 
 7   토요일운영시작시각   3546 non-null   object 
 8   토요일운영종료시각   3546 non-null   object 
 9   공휴일운영시작시각   3546 non-null   object 
 10  공휴일운영종료시각   3546 non-null   object 
 11  열람좌석수       3546 non-null   int64  
 12  자료수(도서)     3546 non-null   int64  
 13  자료수(연속간행물)  3546 non-null   int64  
 14  자료수(비도서)    3546 non-null   int64  
 15  대출가능권수      3546 non-null   int64  
 16  대출가능일수      3546 non-null   int64  
 17  소재지도로명주소    3546 non-null   object 
 18  운영기관명       2826 non-null   object 
 19  도서관전화번호     3337 non-null  

Index(['도서관명', '시도명', '시군구명', '도서관유형', '휴관일', '평일운영시작시각', '평일운영종료시각',
       '토요일운영시작시각', '토요일운영종료시각', '공휴일운영시작시각', '공휴일운영종료시각', '열람좌석수', '자료수(도서)',
       '자료수(연속간행물)', '자료수(비도서)', '대출가능권수', '대출가능일수', '소재지도로명주소', '운영기관명',
       '도서관전화번호', '부지면적', '건물면적', '홈페이지주소', '위도', '경도', '데이터기준일자', '제공기관코드',
       '제공기관명'],
      dtype='object')

In [10]:
from pathlib import Path
import pandas as pd

# 1) 자동 감지해서 읽는 함수 (이미 있는 것 그대로 사용)
def sniff_and_read(path, nbytes=4000):
    p = Path(path)
    raw = p.read_bytes()[:nbytes]

    bom = {
        b'\xff\xfe': 'utf-16le',
        b'\xfe\xff': 'utf-16be',
        b'\xef\xbb\xbf': 'utf-8-sig'
    }
    for sig, enc in bom.items():
        if raw.startswith(sig):
            return pd.read_csv(p, encoding=enc, sep=None, engine="python")

    candidates = ["utf-8", "cp949", "euc-kr", "utf-16", "utf-16le", "utf-16be", "latin1"]
    for enc in candidates:
        try:
            return pd.read_csv(p, encoding=enc, sep=None, engine="python")
        except Exception:
            pass

    raise RuntimeError("encoding sniff 실패")

# 2) 파일 경로
info_path = Path("../data/raw/전국공공도서관정보.csv")
std_path  = Path("../data/raw/전국도서관표준데이터.csv")

# 3) 로드
df_info = sniff_and_read(info_path)
df_std  = sniff_and_read(std_path)

# 4) UTF-8로 재저장
info_utf8_path = info_path.with_name("전국공공도서관정보_utf8.csv")
std_utf8_path  = std_path.with_name("전국도서관표준데이터_utf8.csv")

df_info.to_csv(info_utf8_path, index=False, encoding="utf-8-sig")
df_std.to_csv(std_utf8_path, index=False, encoding="utf-8-sig")

print("[DONE]")
print(f"- saved: {info_utf8_path}")
print(f"- saved: {std_utf8_path}")

[DONE]
- saved: ../data/raw/전국공공도서관정보_utf8.csv
- saved: ../data/raw/전국도서관표준데이터_utf8.csv
